In [45]:
import telebot
from telebot import types
import pandas as pd
import numpy as np
import json
import re
import pyzbar
from pyzbar.pyzbar import decode
from PIL import Image
from nalog_python import NalogRuPython

In [46]:
with open('config/conf.json', 'r') as f:
    conf = json.load(f)

# Создаем экземпляр бота
bot = telebot.TeleBot(conf['token'])

In [47]:
# обработка приветствия
def hello():
    h = ['Dzien dobry', 'Гамарджоба', 'Hello', 
         'Aloha',  'Здравствуйте','Shalom', 'Привет', 'Buenas dias', 'Guten Tag', 'Здраво']
    i = np.random.randint(0,len(h))
    return h[i]

In [48]:
# на вход принимает и обрабатывает json, dict для добавления в таблицу
def purch(x):
    df = pd.read_csv(conf['df_url'], sep='\t')
    purch = pd.json_normalize(x, record_path =['items'])
    purch['check'] = x['requestNumber']
    purch['date'] = x['localDateTime'].replace('T', ' ')
    purch['date'] = pd.to_datetime(purch['date'])
    purch['shop'] = x['user']
    purch['price'] = purch['price']/100
    purch['sum'] = purch['sum']/100
    try:
        purch['ndsSum'] = purch['ndsSum']/100# ---
    except:
        pass
    purch.reset_index(drop=True)
    df = pd.concat([df, purch], axis=0)
    df.reset_index(drop = True)
    df.to_csv('df.csv', sep='\t', index=False, encoding='utf-8-sig')

In [49]:
# открывает файл json
def js(name):
    file_name = name
    with open(file_name, 'r', encoding='utf-8') as f:
        x = json.load(f)
    purch(x)

In [50]:
# обработка фото
def photo(src):
    d = decode(Image.open(src))
    qr = d[0].data.decode("utf-8")
    phone = conf['phone']
    qr_code = qr
    client = NalogRuPython(qr_code, phone)# ------
    
    ticket = client.get_ticket()# -----
    a = json.dumps(ticket, indent=4, ensure_ascii=False)
    x = json.loads(a)
    #пересобираю json в dict и отправляю в функцию обработки
    jsn = {}
    jsn['localDateTime'] = x['query']['date']
    jsn['requestNumber'] = x['ticket']['document']['receipt']['requestNumber']
    jsn['user'] = x['ticket']['document']['receipt']['user']
    jsn['shop'] = x['ticket']['document']['receipt']['retailPlace']
    jsn['items'] = x['ticket']['document']['receipt']['items']
    
    purch(jsn)
    return 'ok'


In [51]:
def analytics(call):
    global df_1
    df_1 = pd.read_csv(conf['df_url'], sep='\t')
    bot.send_message(call.message.chat.id, 'Всего потрачено: '+ str(df_1['sum'].sum())+'\nСредняя стоимость одного товара: '+ str(round(df_1['sum'].mean(),1)))
    

In [52]:
# Функция, обрабатывающая команду /start
@bot.message_handler(commands=['start', 'help'])
def start(m, res=False):
    if m.text == '/start':

        keyboard = types.InlineKeyboardMarkup() #клавиатура
        key_analysis = types.InlineKeyboardButton(text='Анализ', callback_data='analysis') #кнопка
        keyboard.add(key_analysis)
        bot.send_message(m.chat.id, 'Я уже кое-что умею 😃',reply_markup=keyboard)

    else:
        bot.send_message(m.chat.id, 'Мне можно прислать файл JSON из приложения налоговой\nили фото чека с QR кодом.\nЯ добавлю ваши покупки в таблицу')
        
    
#Получение сообщений от юзера
@bot.message_handler(content_types=["text"])
def handle_text(message):
    
    if message.text.lower() in ['привет', 'здравствуйте']:
        bot.send_message(message.chat.id, f'{hello()} {str(message.chat.username)}')
        
        
    else:    
        bot.send_message(message.chat.id, '...' * np.random.randint(1,5))# + message.text)


#получение документа(json)
@bot.message_handler(content_types=['document'])
def handle_docs_doc(message):
    try:

        file_info = bot.get_file(message.document.file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        src = r'load/' + file_info.file_path
        with open(src, 'wb') as new_file:
            new_file.write(downloaded_file)
            
        js(name=src)
        
        bot.reply_to(message, "Пожалуй, я сохраню это")
    except Exception as e:
        bot.reply_to(message, e)        
        

        
@bot.message_handler(content_types=['photo'])
def handle_docs_photo(message):
    
    try:
        chat_id = message.chat.id

        file_info = bot.get_file(message.photo[-1].file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        src = r'load/' + file_info.file_path 
        with open(src, 'wb') as new_file:
            new_file.write(downloaded_file)
           
        bot.reply_to(message, photo(src) )
    except Exception as e:
        bot.reply_to(message, e) 


@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    if call.data == "analysis": 
        bot.send_message(call.message.chat.id, 'Посчитаем : )')   
        analytics(call)
        
    else:
        pass
    
    
# Запускаем бота
bot.polling(none_stop=True, interval=0)
 